In [1]:
import os
import email
import numpy as np
import pandas as pd
from nltk.tokenize.regexp import RegexpTokenizer
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import classification_report
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch
from transformers import AutoTokenizer
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pandas as pd

In [2]:
df = pd.read_csv('/content/Electoral_Analysis.csv')
df

,Unnamed: 0,Tweet,Label
0,0,@anjanaomkashyap I am seeing you as future #bj...,0
1,1,#LokSabhaElections2019 \n23rd May 2019 will re...,0
2,2,#LokSabhaElections2019 \n23rd May 2019 will re...,0
3,3,PM Modi creates a new record of being the only...,0
4,4,@abhijitmajumder Appointment of Successor! \n\...,0
...,...,...,...
39944,39944,"@quizzicalguy In this #LokSabhaElections2019, ...",1
39945,39945,@AnumaVidisha @RahulGandhi @ArvindKejriwal whe...,1
39946,39946,@sherryontopp Early morning #political move af...,1
39947,39947,@mayankgandhi04 @mallesh_2004 Early morning #p...,1


In [4]:
df.dropna(subset = ['Tweet', 'Label'], inplace = True)
df

,Unnamed: 0,Tweet,Label
0,0,@anjanaomkashyap I am seeing you as future #bj...,0
1,1,#LokSabhaElections2019 \n23rd May 2019 will re...,0
2,2,#LokSabhaElections2019 \n23rd May 2019 will re...,0
3,3,PM Modi creates a new record of being the only...,0
4,4,@abhijitmajumder Appointment of Successor! \n\...,0
...,...,...,...
39944,39944,"@quizzicalguy In this #LokSabhaElections2019, ...",1
39945,39945,@AnumaVidisha @RahulGandhi @ArvindKejriwal whe...,1
39946,39946,@sherryontopp Early morning #political move af...,1
39947,39947,@mayankgandhi04 @mallesh_2004 Early morning #p...,1


In [10]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, f1_score
import pandas as pd
import numpy as np

# Load tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model_path = '/content/drive/MyDrive/Unlearning-Model-Weights/Elect_Bert.pt'
model = torch.load(model_path)

# Assuming new data is in a DataFrame with columns 'texts' and 'labels'
new_data = pd.read_csv('/content/Electoral_Analysis.csv')
new_data.dropna(subset = ['Tweet', 'Label'], inplace = True)

texts = new_data['Tweet'].tolist()
labels = new_data['Label'].tolist()

# Tokenize the new data
inputs = tokenizer(texts, padding=True, truncation=True, max_length=512, return_tensors="pt")

# Convert labels to a tensor
labels_tensor = torch.tensor(labels)

# Create a DataLoader
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'], labels_tensor)
dataloader = DataLoader(dataset, batch_size=16)

# Move model to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
model.eval()

# Store predictions and true labels
predictions, true_labels = [], []
results = []
# Evaluate the model
with torch.no_grad():
    for batch in dataloader:
        batch = tuple(b.to(device) for b in batch)
        inputs = {'input_ids': batch[0], 'attention_mask': batch[1], 'labels': batch[2]}
        outputs = model(**inputs)
        logits = outputs.logits
        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].to('cpu').numpy()
        predictions.append(np.argmax(logits, axis=1))
        true_labels.append(label_ids)

# Flatten the predictions and true labels lists
predictions = np.concatenate(predictions, axis=0)
true_labels = np.concatenate(true_labels, axis=0)

for j in range(inputs['input_ids'].size(0)):
  tweet_text = tokenizer.decode(inputs['input_ids'][j], skip_special_tokens=True)
  print(f"Tweet: {tweet_text}\nPredicted Label: {predictions[j]}, Actual Label: {labels[j]}\n")
  results.append({"Tweet": tweet_text, "Predicted Label": predictions[j], "Actual Label": labels[j]})

# Calculate the performance metrics
accuracy = accuracy_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions, average='weighted')  # adjust the average parameter as needed
results_df = pd.DataFrame(results)
print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')
results_df.to_csv('Model-Inference-Performance.csv')

print("results computed..")

Tweet: # loksabhaelections2019 appeal to every voter in voting areas : must come out to vote. you may vote for anyone of the following : 1. you may vote for bjp 2. you may vote for modi 3. you may vote kamal nissan voting for anyone else would be equal to wasting your vote.
Predicted Label: 0, Actual Label: 0

Tweet: modi can say anything because he don ’ t understand anything. # narendramodi # leftalternative # rahulgandhi # cpim
Predicted Label: 0, Actual Label: 0

Tweet: people of # westbengal will lose faith on @ bjp4india if they get ignored and left dying from ongoing violence and no mention from top cadre leader is really forcing # bjp supporters to take side of # tmchhi @ supriyobabul @ narendramodi @ amitshah @ kailashonline @ tajinderbagga # ghatal
Predicted Label: 0, Actual Label: 0

Tweet: @ gyan _ priy22 @ imkhannnn @ wajskhan @ narendramodi @ imrankhanpti @ officialdgispr after 23rd terroristan is finished. # aayegatohmodihi # bjp
Predicted Label: 0, Actual Label: 0

Twee